In [ ]:
# pip install langchain-mcp-adapters
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

OPENAI_API_KEY = 'sk-sdcxstsuwiefzutgkridojrlcovgaggxddyvaicqwynpxebq'
llm = ChatOpenAI(model='Qwen/Qwen2.5-32B-Instruct',
            api_key=OPENAI_API_KEY,
            base_url='https://api.siliconflow.cn/v1',
            streaming=True,
            temperature=0,
            request_timeout=120,
            max_retries=0
            )

In [ ]:
# def tool() -> None:
#     """Testing tool."""
#     print("This is a test tool.")
    
# tools = [tool]

In [ ]:
client = MultiServerMCPClient(
    {
        "math": {
            "command": "python",
            # Replace with absolute path to your math_server.py file
            "args": ["/path/to/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            # Ensure your start your weather server on port 8000
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http",
        }
    }
)
tools = await client.get_tools()

In [ ]:
agent = create_react_agent(
    llm,
    tools=tools,
)

In [ ]:
# from IPython.display import Image, display 
# display(Image(agent.get_graph().draw_mermaid_png()))

In [ ]:
response = await agent.ainvoke({"messages": [{"role": "user", "content": "what is the weather in sf"}]})
async for chunk in agent.astream(
    {"messages": [{"role": "user", "content": "北京天气如何"}]},
    # stream_mode=["updates", "messages", "custom"]
    stream_mode=["messages", "custom"]
):
    try:
        print(chunk[1][0].content, end='', flush=True)
    except Exception as e:
        print(chunk)